In [0]:
# Notebook parameters

params = {
    "refresh_ai_pred": "",
}

# create text widgets
for k in params.keys():
    dbutils.widgets.text(k, "", "")

# fetch values
for k in params.keys():
    params[k] = dbutils.widgets.get(k)
    print(k, ":", params[k])

In [0]:
if params["refresh_ai_pred"] != '1' and spark.catalog.tableExists("4_prod.pacs_ai.predicted_pacs_examcode"):
    dbutils.notebook.exit("AI pred not refreshed.")

In [0]:
%sql

CREATE OR REPLACE TABLE 4_prod.pacs_ai.predicted_pacs_examcode
AS
WITH RawExamCodes AS (
  SELECT RawExamCode
  FROM `4_prod`.pacs.intmd_pacs_examcode
  WHERE RawExamCode LIKE '% %'
  LIMIT 10 -- Adjust the limit as needed
)
SELECT vs.*,
ai_query(
    "azure_openai_gpt4o",
    "Context:\n" || vs.context || "\nQuestion: Choose the best short code for '" || vs_input || "'. In the first line, output the best short code if found. Otherwise output Unknown. In the second line, output the reason. Please note: +C means contrast. If contrast is not specified then choose a non-contrast short code.",
    failOnError=>false
) AS AIResponse

FROM RawExamCodes rec
JOIN LATERAL (
  SELECT 
    rec.RawExamCode AS vs_input, 
    array_join(collect_set(concat_text), '\n') AS context
  FROM vector_search(
    index => '1_inland.sectra.pacs_examcode_concat_vs_index',
    query_text => rec.RawExamCode,
    num_results => 20
  )
  GROUP BY vs_input
) vs ON vs.vs_input = rec.RawExamCode